In [4]:
import os
import pickle
import json
import numpy as np
import xgboost as xgb

# Load model when the container starts
MODEL_PATH = "/opt/ml/model/model.xgb"

def model_fn(model_dir):
    """Load the trained model"""
    model = xgb.Booster()
    model.load_model(os.path.join(model_dir, "model.xgb"))
    return model

def input_fn(request_body, request_content_type):
    """Parse input data from JSON"""
    if request_content_type == "application/json":
        data = json.loads(request_body)["instances"]
        return np.array(data)
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")

def predict_fn(input_data, model):
    """Make predictions"""
    dmatrix = xgb.DMatrix(input_data)
    predictions = model.predict(dmatrix)
    return predictions.tolist()

def output_fn(predictions, response_content_type):
    """Format output"""
    return json.dumps({"predictions": predictions})


In [7]:
# Save the inference script
script_content = """import os
import pickle
import json
import numpy as np
import xgboost as xgb

# Load model when the container starts
MODEL_PATH = "/opt/ml/model/model.xgb"

def model_fn(model_dir):
    \"\"\"Load the trained model\"\"\"
    model = xgb.Booster()
    model.load_model(os.path.join(model_dir, "model.xgb"))
    return model

def input_fn(request_body, request_content_type):
    \"\"\"Parse input data from JSON\"\"\"
    if request_content_type == "application/json":
        data = json.loads(request_body)["instances"]
        return np.array(data)
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")

def predict_fn(input_data, model):
    \"\"\"Make predictions\"\"\"
    dmatrix = xgb.DMatrix(input_data)
    predictions = model.predict(dmatrix)
    return predictions.tolist()

def output_fn(predictions, response_content_type):
    \"\"\"Format output\"\"\"
    return json.dumps({"predictions": predictions})
"""

# Write to a file
with open("inference.py", "w") as f:
    f.write(script_content)

print("✅ inference.py file created successfully!")


✅ inference.py file created successfully!


In [8]:
import os
print("Does inference.py exist? ➝", os.path.exists("inference.py"))


Does inference.py exist? ➝ True


In [9]:
import boto3

s3_client = boto3.client("s3")
bucket_name = "network-anomaly-dataset-001aefd6"
script_path = "inference.py"
s3_key = "sagemaker/inference.py"  # Define where in S3 to store it

s3_client.upload_file(script_path, bucket_name, s3_key)

print("Inference script uploaded successfully!")


Inference script uploaded successfully!
